In [1]:
!pip install tqdm==4.28.1
!pip install -U gast==0.2.2
!pip install gdown

!gdown https://drive.google.com/uc?id=14CB3Vw4jPf-8-DAriB9XDq4mfbs4o747
!mkdir datasets
!tar -C datasets -xzf airport-alunos.tgz


  Using cached https://files.pythonhosted.org/packages/91/55/8cb23a97301b177e9c8e3226dba45bb454411de2cbd25746763267f226c2/tqdm-4.28.1-py2.py3-none-any.whl
Requirement already up-to-date: gast==0.2.2 in /opt/anaconda3/envs/siamesas/lib/python3.7/site-packages (0.2.2)
  Using cached https://files.pythonhosted.org/packages/b0/b4/a8e9d0b02bca6aa53087001abf064cc9992bda11bd6840875b8098d93573/gdown-3.8.3.tar.gz
  Using cached https://files.pythonhosted.org/packages/93/83/71a2ee6158bb9f39a90c0dea1637f81d5eef866e188e1971a1b1ab01a35a/filelock-3.0.12-py3-none-any.whl
  Created wheel for gdown: filename=gdown-3.8.3-cp37-none-any.whl size=8852 sha256=e0d144f37ddd3acac0953840f5851a2236a72447abff4dbe929c5947a6324d07
  Stored in directory: /home/joao/.cache/pip/wheels/a7/9d/16/9e0bda9a327ff2cddaee8de48a27553fb1efce73133593d066
Successfully built gdown
Downloading...
From: https://drive.google.com/uc?id=14CB3Vw4jPf-8-DAriB9XDq4mfbs4o747
To: /home/joao/Documents/Doutorado/ammd2-siamesas/airport-alunos.t

In [8]:
#%tensorflow_version 2.x
import tensorflow as tf; print(tf.__version__)
import tensorflow.keras
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K


from sklearn.datasets import load_sample_image;

import numpy as np
import tqdm; print(tqdm.__version__)
import matplotlib.pyplot as plt
%matplotlib inline

2.0.0
4.28.1


In [9]:
def base_model(input_shape):
    """
        Model architecture
    """
    
    input_layer = Input(shape = input_shape)
    
    seq_model = Conv2D(8, (3, 3), activation = 'relu')(input_layer)
    seq_model = BatchNormalization()(seq_model)
    seq_model = Activation('relu')(seq_model)
    seq_model = MaxPooling2D((2, 2), padding = 'same')(seq_model)
    
    seq_model = Conv2D(16, (3, 3), strides = (2, 2), padding = 'same')(seq_model)
    seq_model = BatchNormalization()(seq_model)
    seq_model = Activation('relu')(seq_model)
    seq_model = MaxPooling2D((2, 2), padding = 'same')(seq_model)
    
    seq_model = Conv2D(32, (3, 3), strides = (2, 2), padding = 'same')(seq_model)
    seq_model = BatchNormalization()(seq_model)
    seq_model = Activation('relu')(seq_model)
    seq_model = MaxPooling2D((2, 2), padding = 'same')(seq_model)
    
    seq_model = Conv2D(64, (3, 3), strides = (2, 2), padding = 'same')(seq_model)
    seq_model = BatchNormalization()(seq_model)
    seq_model = Activation('relu')(seq_model)
    seq_model = MaxPooling2D((2, 2), padding = 'same')(seq_model)
    
    seq_model = Flatten()(seq_model)
    seq_model = Dense(64)(seq_model)
    
    seq_model = Model(inputs=input_layer, outputs=seq_model)
    
    return seq_model
  
def siamise_model(input_shape):
    
    seq_model = base_model(input_shape)
    
    left_input = Input(shape = input_shape)
    right_input = Input(shape = input_shape)
    
    left_encod = seq_model(left_input)
    right_encod = seq_model(right_input)

    lambda_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    lambda_distance  = lambda_layer([left_encod, right_encod])
    
    return Model(inputs=[left_input,right_input], outputs=lambda_distance)

In [10]:
file_names_treino = !find datasets/airport-alunos/treino -name '???.png' | sort
file_names_val = !find datasets/airport-alunos/val -name '???.png' | sort

print(file_names_treino[:2])
print(file_names_val[:2])

['datasets/airport-alunos/treino/0/000/000.png', 'datasets/airport-alunos/treino/0/000/001.png']
['datasets/airport-alunos/val/0/200/000.png', 'datasets/airport-alunos/val/0/200/001.png']


In [11]:
#load_sample_image('datasets/airport-alunos/treino/0/094/051.png')

#tf.image.decode_image(tf.io.read_file('datasets/airport-alunos/treino/0/094/051.png'))

In [ ]:
from tqdm import tqdm
import random

def load_img(image_file):
    return tf.io.read_file(image_file)

def img_to_array(image_raw):
    return tf.image.decode_image(image_raw)

def person_to_img(file_names):
    cam_path_to_img = {}
    pids = [[], []]
    img_counts = [0, 0]
    for f in list(file_names):
        cid = int(f.split('/')[-3])
        pid = int(f.split('/')[-2])
        iid = int(f.split('/')[-1][:3])
        pids[cid].append((pid, iid))
        cam_path_to_img[cid, pid, iid] = img_to_array(load_img(f))
        img_counts[cid] += 1
    print('Images in cam1 = %d, cam2 = %d'%(img_counts[0], img_counts[1]))
    # dic person_to_img e person ids em cada camera
    return cam_path_to_img, list(set(pids[0])), list(set(pids[1]))

def filter_possible(pid1, pids2, same=True):
    person_number = pid1[0]
    if same is True:
        possible_ids2 = [pid2 for pid2 in pids2 if pid2[0] == person_number]
    else:
        possible_ids2 = [pid2 for pid2 in pids2 if pid2[0] != person_number]
    return possible_ids2

def combine_cam_files(pids1, pids2):
    i = 0
    d = {}
    for pid1 in tqdm(pids1):
        if i % 2 == 0:
            d[pid1] = filter_possible(pid1, pids2, same=True)
        else:
            d[pid1] = filter_possible(pid1, pids2, same=False)
        i += 1
    return d

def get_batch(d_combination, cam_img_dict):
    pairs_labels = []
    i = 0
    for key, value in d_combination.items():
        if i % 2 == 0:
            t = ((0, *key), (1, *random.choice(value)), 1.)
        else:
            t = ((0, *key), (1, *random.choice(value)), 0.)
        i += 1
        pairs_labels.append(t)
    train_data = pairs_labels
    X_a_train = np.array([cam_img_dict[x[0]] for x in train_data], dtype = np.float32)
    X_b_train = np.array([cam_img_dict[x[1]] for x in train_data], dtype = np.float32)
    y_train = np.array([x[2] for x in train_data], dtype = np.float32)
    return X_a_train, X_b_train, y_train

# carrega imagens para memória
cam_img_dict_val, pids1_val, pids2_val = person_to_img(file_names_val)

# gera listas de similares e dissimilares na memória
d_combination_val = combine_cam_files(pids1_val, pids2_val)

# obtem pares de batches apartir das imagens em memória
X_a_val, X_b_val, y_val = get_batch(d_combination_val, cam_img_dict_val)

 13%|█▎        | 374/2824 [00:00<00:01, 1631.56it/s]

Images in cam1 = 2824, cam2 = 3648


100%|██████████| 2824/2824 [00:01<00:00, 1698.64it/s]


In [ ]:
# plot pics side by side
def show_side_by_side(figs, titles = None, limit = 10, figsize=(20, 4), cmap = 'gray', grid = False):
    minval = min(limit, figs.shape[0])
    plt.figure(figsize = figsize)
    for i in range(minval):
        subplot = plt.subplot(1, limit, i + 1)
        extent = (0, figs[i].shape[1], figs[i].shape[0], 0)
        subplot.imshow(figs[i], cmap = cmap, extent = extent)
        if titles:
            subplot.set_title(titles[i])
        if grid:
            subplot.grid(color='gray', linestyle='-', linewidth=1)
        else:
            subplot.get_xaxis().set_visible(False)
            subplot.get_yaxis().set_visible(False)
    plt.show()
    
def plot_sample(cam1, cam2, val):
    rindices = np.random.choice(np.array(range(cam1.shape[0])), 10, replace=False)
    show_side_by_side(cam1[rindices], titles = [('DIF' if n == 0 else 'IGUAL') for n in val[rindices]])
    show_side_by_side(cam2[rindices])
    plot_sample(X_a_val/255, X_b_val/255, y_val)